# Lab 1 - Batch & ETL processing of Big Data with Spark SQL

AdventureWorks is an e-commerce retailer who is looking to improve how they manage the data assets produced by their platform. As a starting point they would like collect their data in a manner that enables easier exploration and prepares the data for downstream analytics processes that can yield new insights. Adventurworks has asked you to process and prepare their flat file data into a tabular format that offers better query perfomrance and can be queried using SQL.

In the lab you will learn how to use Spark SQL (and PySpark) to batch process a 10GB text file dataset, quickly explore its content, identify issues with the data, clean and format the data and load it into Hive tables to support downstream analytics.

## Pre-requisites
Before attempting this lab, make sure you:
* Have provisioned an HDInsight 3.6 cluster with Spark 2.1 
* Have copied the retaildata to the default storage for your Spark cluster.
* Are running these notebooks from your HDInsight cluster.
These steps are described in the lab-preqs guide included with these notebooks.

## Understand the source data

Let's start by examing the weblogs included in the retaildata dataset. Each days worth of data is stored under a folder path of month/day in a file called weblog.txt as the following output illustrates. The files sizes (the third column) are in expressed as M for MB.

In [ ]:
%%sh
hdfs dfs -ls -h /retaildata/rawdata/weblognew/1/*

Now, let's import the Python modules and functions we will use in this notebook.

In [ ]:
import pprint, datetime
from pyspark.sql.types import *
from pyspark.sql.functions import date_format,unix_timestamp

Let's examine the header row along with one row data of the weblog files to get a sense for the schema.

In [ ]:
lines = spark.read.text("/retaildata/rawdata/weblognew/1/1/weblog.txt")
lines.show(2, False)

That's enough to give us a sense of what these files contain. Let's move on to trying to parse these files.

## Prepare and stage the data

First, we will try to read the data directly as a CSV, letting Spark infer the schema.

In [ ]:
# TODO: invoke the spark.read.csv function with a path to a single weblog.txt. 
df = spark.#Complete this line# 
df.dtypes

Clearly a dataset with one column of type string is not what we want. If you look at the lines we outputted above, it should be make sense why we got this. The weblog data has one row per event, where each column in a row is separated by a pipe (|) character. 

Let's see how we can address this.

## Handle input format issues. 

Here we will read the file inferring the schema from the data, and tell Spark to use the first row to provide the column names. See the [documentation for the csv()](http://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader) function in the DataFrameReader class.

In [222]:
# TODO: 1. modify your path to the weblogs so that it uses a wildcard (*) to 
# include all months and all days for the data available, 
# TODO 2. use the sep parameter to indicate the row seperator character
# TODO 3. use the header parameter to treat the first row of each files as the column names  
df = spark.read.csv(#TODO 1,#TODO 2,#TODO 3)

Let's take a peek at the data.

In [ ]:
df.show(5)

Ok, so far so good. What was the schema that Spark inferred? Run the following to find out.

In [ ]:
pprint.pprint(df.dtypes)

Notice that above will treat all columns as strings. 
However, we have columns that should be 
   * numeric: UserId, ProductID, Quantity, Price, TotalPrice, PageStopDuration
   * string: TransactionDate (we'll worry about treating this as a timestamp data type later)
   
Let's fix that by providing a schema. See [this description of creating a StructField](http://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#pyspark.sql.types.StructField) and see [this link](http://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#pyspark.sql.types.DataType) for a list of data types. 



In [ ]:
weblogs_schema = StructType([
        StructField('UserId',LongType(),False), 
        StructField('SessionId', StringType()), 
        StructField('ProductId', IntegerType()), 
        StructField('Quantity', IntegerType()), 
        StructField('Price', DoubleType()), 
        StructField('TotalPrice', DoubleType()), 
        StructField('ReferralURL', StringType()), 
        StructField('PageStopDuration', IntegerType()), 
        StructField('Action', StringType()), 
        StructField('TransactionDate', StringType())])

Next we will use this schema as an input to the csv function and then examine the new schema.

In [ ]:
# TODO: provide the weblogs_schema as the value for the schema parameter in the call to csv().
df = spark.read.csv("/retaildata/rawdata/weblognew/*/*/weblog.txt",
                    #TODO,
                    sep="|",
                    header=True
                    )
pprint.pprint(df.dtypes)

Let's confirm the data still looks good with this new schema.

In [ ]:
df.show(5)

## Clean and format the data

Earlier we glossed over treating the TransactionDate field as a timestamp type, choosing to leave it as string type instead. Let's work to fix that. First, let's look at some sample values to get a sense of the shape of the TransactionDate data.

In [ ]:
df.select("TransactionDate").show(200)

As you can see in the above listing of transaction dates, the timestamp is represented in a human friendly way, but it is not the ideal format for use when querying the data. We would prefer to see this TransactionDate in the form:

`2016-01-02 22:05:55`



The pyspark.sql.functions module contains a unix_timestamp function that might help us with the parsing of the strings in the TransactionDate column.

Lets read the documentation (aka the docstring) about the function. All functions in the PySpark API are documented this way, so you can use this same approach to learn about any new function you encounter.

In [ ]:
print(unix_timestamp.__doc__)

We can use the unix_timestamp inside of select to operate on the column of interest.

In [ ]:
# TODO: Author the format string to parse the source data.
df.select(
    "TransactionDate",
    unix_timestamp("TransactionDate", #TODO).alias("NewDate").cast("timestamp")
).show(5, False) 

So far so good. The only issue is the values in NewDate appear to have a fractional second component like 00:24:00**.0** that we do not want. So it looks like we now need to format the timestamp to use our preferred format. 

The date_format function can help us with this. Let's look at the documentation for it.

In [ ]:
print(date_format.__doc__)

The date_format function relies on a format string pattern. You need to provide a custom format string for the timestamps used by the weblogs dataset, can you figure out the string? Hint: under the covers Spark uses the SimpleDateFormat class to format dates, see the [SimpleDateFormat](http://docs.oracle.com/javase/6/docs/api/java/text/SimpleDateFormat.html) documentation. 

Test out your format string on the following example date to make sure you get it right. If your are getting a null value back from date_format, that means it was unable to parse the input using the pattern you supplied (and so there's something still not quite right with your pattern).

In [ ]:
# TODO: Format the timestamp by providing a format string that yields values like 2016-01-01 14:10:00.
df.select(
    "TransactionDate",
    date_format(
        unix_timestamp("TransactionDate", "M/d/yyyy h:mm:ss a").cast("timestamp"),
        #TODO).alias("NewDate")
).show(5, False) 

Once you think you have the date formatting figured out, it's time to try it against the whole dataset to see how it works. Remember, there are two potential issues here. Your format string may not be quite right OR the data itself might be garbage data. 

The following query runs across all of the data to find the instances where we were unable to parse and reformat the date. 

**Note:** this query will take about 4 minutes to complete since it is running across the entire dataset.

In [ ]:
pprint.pprint(
    df.select("TransactionDate",
          date_format(
            unix_timestamp("TransactionDate","M/d/yyyy h:mm:ss a").cast("timestamp"),
            "yyyy-MM-dd HH:mm:ss").alias("date")
         ).where("date IS NULL").take(5)
)

It looks like there are 3 rows that have TransactionDates that we could not parse. Rather than try to fix these values, we will just delete the rows from the cleaned dataset. 

In the following we apply a where clause to filter out the above three rows whose TransactionDate could not be parsed/formatted.

In [ ]:
# TODO: Complete the SQL where filter clause to eliminate rows where CleanedTransactionDate is NULL
df2 = df.select("*",
          date_format(
            unix_timestamp("TransactionDate","M/d/yyyy h:mm:ss a").cast("timestamp"),
            "yyyy-MM-dd HH:mm:ss").alias("CleanedTransactionDate")
         ).where(#TODO)

The date formatting for the new columns is in place, but let's take a look at the schema that resulted.

In [ ]:
df2.printSchema()

Apparently, while the value of CleanedTransactionDate is now better formatted, it is still being treated as having a string data type. We can fix that by casting the new column to be of type ```timestamp```

In [ ]:
cleaned_df = df.select("*",
          date_format(
            unix_timestamp("TransactionDate","M/d/yyyy h:mm:ss a").cast("timestamp"),
            "yyyy-MM-dd HH:mm:ss").cast("timestamp").alias("CleanedTransactionDate")
         ).where("CleanedTransactionDate IS NOT NULL")

Let's examine the resulting schema

In [ ]:
cleaned_df.printSchema()

Now our CleanedTransactionDate column will actually use the timestamp date type.

## Create a view to explore the data using SQL without moving the data

Now we will create a temporary view that enables us to use SQL statements to query our cleaned up dataset.

In [ ]:
# TODO: Invoke the createOrReplaceTempView function on your cleaned_df object 
# to create a view named "weblogs_view".
cleaned_df.#TODO( )

We use the ```%%sql``` cell magic to switch the mode of the cell from running Python to running SQL. Everything on the lines below ```%%sql``` is SQL that SparkSQL will execute for us.

In [ ]:
%%sql
SELECT * FROM weblogs_view limit 10

## Export the prepared data to persistent Hive tables 

The above view is temporary and will eventually be deleted. If we want our cleansed data to remain permanently, we should export it to a Hive table. 

Run the following to copy the data from source location into the Hive warehouse location. This will create a Hive managed table you can query with Hive as well as with Spark. If you get an error in the below, check your schema and the parameters you passed to the csv function above.

In [ ]:
#TODO 1: Set the mode to overwrite so when this cell is re-run it re-creates the table and data.
#TODO 2: Use the saveAsTable method to create a Hive table called "weblogs".
cleaned_df.write.mode(#TODO 1).#TODO 2("weblogs")

Run the following to see how files behind the Hive table land in Azure Storage.

In [ ]:
%%sh
hdfs dfs -ls -h /hive/warehouse/weblogs

Observe that our source data, which was originally stored in text files, is now stored as a series of parquet files. 

The Hive table by default is saved out in the Parquet format, which as we'll see momentarily transparently provides some performance improvement over querying the raw text data. 

## Verify you can query the Hive table using Spark.

We can query a Hive table using Spark SQL in the familiar way. Let's start by checking how many rows our new Hive table contains.

In [ ]:
#TODO: Author a SQL query that counts the number of rows in the weblogs table
spark.sql(#TODO).show()

Let's also spot check the date ranges it encompasses.

In [ ]:
%%sql
SELECT Max(CleanedTransactionDate), Min(CleanedTransactionDate) FROM weblogs

In [ ]:
%%sql
SELECT * FROM weblogs LIMIT 5

Did you notice how much faster the queries seemed to run against our Hive table? This was all the result of saving the table's data to Parquet.

## Prepare the Users table

AdventureWorks also has a users dataset in flat files that provides details about the users whose events are captured in the weblogs. You will need to prepare this table and create a Hive table for this data just you did for the weblogs.

In this case, however, you will take a different approach to parsing the data. It's not always the case that the source data is shaped the way you need, and you need to apply parsing beyond that supplied by the spark.read.csv function. In this case, we need to handle the parsing of certain columns of data by applying numeric and timestamp parsing, but this approach generalizes to any data set where you need to shape the data field by field and line by line.

Begin by examining one row of the data to get a sense of the shape. Note that these files do not include a header row.

In [ ]:
lines = spark.read.text("/retaildata/rawdata/UserFile/")
lines.take(1)

Looking at the above output DataFrame, observe that the one eaxmple line of the file contains comma separated values. What's more, none of the invidual values within a column appear to have commas that could throw off any simple parsing. 

Observe that there are 20 fields in the row of data, where each field is separated from the next by a comma.

Parse the files by splitting each line into fields by splitting on a "," and create as output a collection of python tuples. Note that you will need to access the underlying RDD of the DataFrame to do this.

In [ ]:
fields = lines.rdd.map(lambda l: l.value.split(","))
rows = fields.map(lambda p: (p[0], p[1], p[2], p[3], p[4], p[5], 
                            p[6], p[7], p[8], p[9], p[10], 
                            p[11], p[12], p[13], p[14], p[15],
                            p[16], p[17], p[18], p[19]))

Run the following and confirm that the data appears to be split correctly for the first row.

In [ ]:
rows.first()

The following cell provides the schema of the data for the data, as provided by AdventureWorks. In the cell that follows, you will parse and load the rows of the data line by line using a Python lambda function and then apply this schema.

In [ ]:
users_schema = StructType([
        StructField('id',IntegerType(),False), 
        StructField('LoginMd5', StringType()), 
        StructField('Email', StringType()), 
        StructField('FirstName', StringType()), 
        StructField('PictureLarge', StringType()), 
        StructField('LastName', StringType()), 
        StructField('LoginSha1', StringType()), 
        StructField('Username', StringType()), 
        StructField('Title', StringType()), 
        StructField('Gender', StringType()), 
        StructField('LoginSalt', StringType()),
        StructField('Phone', StringType()),
        StructField('Password', StringType()),
        StructField('LoginSha256', StringType()),
        StructField('PictureThumbnail', StringType()),
        StructField('Age', IntegerType()),
        StructField('Cell', StringType()),
        StructField('BirthDate', TimestampType()),
        StructField('Registered', TimestampType()),
        StructField('PictureMedium', StringType())
    ])

Try to apply the schema to the rows as you have them now. What is the error? **HINT: You are looking for a TypeError**

In [ ]:
#TODO: call the createDataFrame method on the spark object, providing the rows you parsed 
# and their schema
users_df = spark.#TODO( , )
users_df.take(1)

In the schema provided by Adventurworks most of the fields are strings, except for:
* id which should be an integer
* age which should be an integer
* birthdate which should be a timestamp
* registered which whould be a timestamp

Looking at our example row above, we see an example value for each:
* id: '9858'
* Age: '53'
* BirthDate: '1964-11-24 10:54:00.000'
* Registered: '2016-10-11 07:38:00'

In the cell below, complete the Python parsing code so that each of string values is parsed succesfully. For the timestamp values, refer to [this table for the format codes](https://docs.python.org/2/library/datetime.html#strftime-strptime-behavior).

In [223]:
#TODO 1: parse an int from the string id
#TODO 2: parse an int from the string age
#TODO 3: provide the format string to parse a timestamp from the string BirthDate 
#TODO 4: provide the format string to parse a timestamp from the string Registered
print(
    #TODO 1('9876'),
    #TODO 2('53'),
    datetime.datetime.strptime('1964-11-24 10:54:00.000', #TODO 3),
    datetime.datetime.strptime('2016-10-11 07:38:00', #TODO 4)
    )

9876 53 1964-11-24 10:54:00 2016-10-11 07:38:00

Next, modify the lambda to apply your parsing function to the appropriate cells. 

In [ ]:
##TODO 1: parse the age field into an int
#TODO 2: parse the Registered field into a timestamp
fields = lines.rdd.map(lambda l: l.value.split(","))
rows = fields.map(lambda p: (int(p[0]), p[1], p[2], p[3], p[4], p[5], 
                            p[6], p[7], p[8], p[9], p[10], 
                            p[11], p[12], p[13], p[14], #TODO 1(p[15]),
                            p[16], 
                            datetime.datetime.strptime(p[17], "%Y-%m-%d %H:%M:%S.%f"), 
                            #TODO 2(p[18], ),
                            p[19]))

Now create a new DataFrame and verify your first row was handled correctly.

In [ ]:
users_df = spark.createDataFrame(rows, users_schema)
users_df.take(1)

As before, create a temporary view for this DataFrame.

In [ ]:
users_df.createOrReplaceTempView("users_view")

Next, query the view to summarize the table and spot check your data parsing is working correctly.

In [ ]:
%%sql
select count(id), min(id), max(id), 
        min(age), max(age), min(BirthDate), max(BirthDate), 
        min(Registered), max(Registered)
from users_view

Finally, save this view as new Hive table called users.

In [ ]:
users_df.write.mode("overwrite").saveAsTable("users")

## Cleanup

Since we don't need the weblogs_view anymore, we can drop it. Because it is a temporary view it will get dropped when our Spark session ends, but we can use the following to drop it sooner.

In [ ]:
spark.catalog.dropTempView("weblogs_view")

## Conclusion

In the lab you have learned how to use Spark SQL (and PySpark) to batch process a 10GB text file dataset, quickly explore its content, identify issues with the data, clean and format the data and load it into a Hive table to support downstream analytics. Specifically you:
* Loaded text flat files into a Spark SQL DataFrame.
* Added a new column to the DataFrame that stored the TransactionData in the format desired.
* Created a temporary view from the DataFrame.
* Copied the data from the DataFrame into a Hive table and stored the data using Parquet.
* Verified you can query the Hive table and observed the increase in query speed that came from saving the data in Parquet.